### Read documentation:

http://scipy.github.io/old-wiki/pages/Cookbook/FittingData
http://scipy-cookbook.readthedocs.io/items/FittingData.html

http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.rv_continuous.fit.html

http://docs.scipy.org/doc/scipy/reference/optimize.html

http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.leastsq.html

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import optimize
import matplotlib.pyplot as plt
#import warnings

In [2]:
from Matrix import Matrix
from TLD import *
from AuxFunctions import *

In [3]:
import os

In [4]:
dist_cont = [d for d in dir(stats) if
             isinstance(getattr(stats,d), stats.rv_continuous) and
             hasattr(getattr(stats,d), 'fit')]

In [5]:
#log of distributions not working properly
dist_cont_not_working = ['levy_stable']

In [6]:
dist_cont_working = list(set(dist_cont) - set(dist_cont_not_working))
#dist_continu_working

In [7]:
#TODO: CHECK is Tanner area == !?
def Tanner_func(x, alpha, beta):
    return (x ** beta) * np.exp(alpha * x)

In [8]:
#TEMPORARY: fit method does not work as expected.
class Tanner(stats.rv_continuous):
    def _pdf(self, x, alpha, beta):
        return (x ** beta) * np.exp(alpha * x)
    def _stats(self, alpha, beta):
        #starting values for fit
        return 0., 0., 0., 0.

In [9]:
tanner = Tanner(name='tanner', shapes='alpha, beta')

In [10]:
#tanner.fit(Ynorm) #TMP don't re-run this cell. it's for demonstration only.
(0.00064354474335964512,
 4.1048820704712746e-08,
 -0.00019229306301766426,
 0.75015919468560166)

(0.0006435447433596451,
 4.1048820704712746e-08,
 -0.00019229306301766426,
 0.7501591946856017)

In [11]:
class Tanner(stats.rv_continuous):
    def _pdf(self, x, alpha, beta):
        return (x ** beta) * np.exp(alpha * x)
    def _stats(self, alpha, beta):
        #starting values for fit
        return 0., 0., 0., 0.
    #TODO: Review. It would be ideal if Tanner could use built in fit method,
    #      but this implementation works similarly. Wath ydata.index.values 
    def fit(self, ydata, p0=[0.15,-0.9], *args, **kwargs):
        return optimize.curve_fit(self._pdf, ydata.index.values, ydata, *args, **kwargs)[0]

In [12]:
tanner = Tanner(name='tanner', shapes='alpha, beta')
stats.tanner = tanner #as if it had always been in scipy.stats

In [13]:
#TMP don't re-run this cell. it's for demonstration only.
#tanner.fit(Ynorm, p0=[0.15,-0.9])
np.array([-3.39581168,  6.73004206])

array([-3.39581168,  6.73004206])

In [14]:
#TMP don't re-run this cell. it's for demonstration only.
#optimize.curve_fit(Tanner_func, X, Ynorm, p0=[0.15,-0.9])
(np.array([-3.39579654,  6.72999835]), np.array([[ 0.16630225, -0.47650378],
        [-0.47650378,  1.38303769]]))

(array([-3.39579654,  6.72999835]), array([[ 0.16630225, -0.47650378],
        [-0.47650378,  1.38303769]]))

In [15]:
#src: http://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python
def fit_distribs(TLD, distrib_names, *args, **kwargs):
    '''Returns a dictionary of distributions fitted for each column in TLD.
    Drops NaN values.'''
    
    if isinstance(distrib_names, str):
        distrib_names = list(distrib_names)
    elif not isinstance(distrib_names, list):
        raise ValueError("distrib_names must be a distirbution name or a list.")
    
    distribs = {}
    for col in TLD:
        distribs[col] = [getattr(stats, fn)(
                            *getattr(stats, fn).fit(TLD[col].dropna(), *args, **kwargs))
                         for fn in distrib_names]
    return distribs

In [16]:
def fit_distribs_to_df(distrib_dict: dict, xdata: pd.Series) -> pd.DataFrame:
    '''Returns a dataframe with all the distributions in distrib_dict applied to xdata.'''
    df = pd.DataFrame.from_dict(
    {(k, d.dist.name): d.pdf(xdata) for k,dlst in distrib_dict.items() for d in dlst})
    df.index = xdata
    return df

In [17]:
def append_distribs_to_df(TLD, distrib_names, level=0, include_ydata=True,
                          *args, **kwargs):
    '''Returns TLD expanded with distrib_names fitted for each column.
    xdata is taken from TLD index level as specified by level.'''
    TLDdistribs = fit_distribs(TLD, distrib_names)
    df = fit_distribs_to_df(TLDdistribs, TLD.index.get_level_values(level))
    if include_ydata:
        for col in TLD:
            df.loc[:,(col,'ydata')] = TLD[col]
        df.sort_index(axis=1, level=0, inplace=True)
    return df

In [18]:
#TODO
def n_best_fitting_distrib(data, distrib_names, n, *args, **kwargs):
    '''returns the best n fitting distributions of the specified
    list of distirbutions.'''
    ...

In [19]:
def lognorm_mu_sigma(params):
    '''Returns the lognorm parameters (mu, sigma) from params
    estimated from scipy.stats.lognorm.fit(obs_data)'''
    shape, loc, scale = params
    mu, sigma = np.log(scale), shape
    return mu, sigma

# Dev

In [20]:
tldf = os.path.join('example_data', 'obsTLD.txt')
tld = pd.read_csv(tldf, sep=', ', engine='python')

In [21]:
tld['purp'] = tld['Purpose'] + '_' + tld['Direction']
tld = tld[['purp', 'Trips', 'StageDistance']]
tld = tld.groupby(['purp','StageDistance']).sum()['Trips'].unstack(1).T

In [22]:
tld = TLD(tld)

tld = tld.band_agg(2, upper_band=False)
tld = tld.mid_band()
tld.set_zero()
tld = tld.sort_index()
tld = tld.truncate(100)

tld

purp,Business_FH,Business_NHB,Business_TH,Commuting_FH,Commuting_TH,Other_FH,Other_NHB,Other_TH
0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1.0,165.13326,202.74756,157.46388,1357.71311,1190.05246,7919.14051,3037.07764,7847.08802
3.0,226.95261,358.50580,196.02925,2368.38976,2149.93329,8655.09300,3471.22347,8747.68704
5.0,150.37551,248.97454,155.68414,1372.53063,1261.14605,4421.69575,1861.72677,4528.84577
7.0,123.83953,149.03070,121.85702,1114.48384,1022.47393,2782.01956,1239.66604,2997.33634
9.0,75.05621,97.81700,71.47052,811.83240,740.28069,1726.36318,775.11899,1837.13827
11.0,67.96856,130.53456,60.22266,799.88807,712.38759,1299.80091,711.78903,1371.52428
13.0,54.91195,95.21575,52.11346,555.12178,509.35058,775.13014,469.83627,927.48160
15.0,67.69446,77.39393,62.61928,508.14491,432.28882,739.40094,397.19879,756.07403
17.0,56.60834,68.34092,61.34358,410.53220,365.00327,415.56240,210.48282,441.91608


In [23]:
OutputName = os.path.join('example_outputs', 'TLD.png')
tld.to_JPG(OutputName)

In [24]:
Y = tld['Business_FH'].dropna()
Y = tld['Business_FH'].fillna(0)
X = Y.index.values
Ynorm = Y.apply(lambda x: x/Y.sum())

In [25]:
tldn = tld.norm

In [26]:
distrib_names = 'lognorm gamma exponnorm exponpow tanner'.split()
TLDdistribs_df = append_distribs_to_df(tldn, distrib_names)#, include_ydata=False)
TLDdistribs_df = TLD(TLDdistribs_df)
TLDdistribs_df

Business_FH                                                            \
          exponnorm       exponpow          gamma       lognorm        tanner   
0.0    4.729591e+01   1.298744e+17   3.783752e+11  2.822240e+01  0.000000e+00   
1.0    8.324658e-20   3.904250e-12   1.721573e-13  3.614296e-04  2.721821e+00   
3.0    2.532712e-61  6.009565e-110   9.522951e-41  3.337038e-06  2.016412e+01   
5.0   7.705579e-103   0.000000e+00   6.570501e-68  2.916631e-07  1.493822e+02   
7.0   2.344362e-144   0.000000e+00   4.840564e-95  5.354382e-08  1.106670e+03   
9.0   7.132538e-186   0.000000e+00  3.682130e-122  1.441005e-08  8.198558e+03   
11.0  2.170019e-227   0.000000e+00  2.854775e-149  4.910193e-09  6.073747e+04   
13.0  6.602114e-269   0.000000e+00  2.241476e-176  1.965459e-09  4.499622e+05   
15.0  2.008641e-310   0.000000e+00  1.775855e-203  8.846661e-10  3.333460e+06   
17.0   0.000000e+00   0.000000e+00  1.416479e-230  4.354751e-10  2.469531e+07   
19.0   0.000000e+00   0.000000e+00  1.135765e-257  2.300547e-10  1.829506e+08   
21.0   0.000000e+00   0.000000e+00  9.145059e-285  1.286978e-10  1.355355e+09   
23.0   0.000000e+00   0.000000e+00  7.388772e-312  7.549217e-11  1.004089e+10   
25.0   0.000000e+00   0.000000e+00   0.000000e+00  4.608667e-11  7.438603e+10   
27.0   0.000000e+00   0.000000e+00   0.000000e+00  2.911161e-11  5.510749e+11   
29.0   0.000000e+00   0.000000e+00   0.000000e+00  1.893961e-11  4.082534e+12   
31.0   0.000000e+00   0.000000e+00   0.000000e+00  1.264366e-11  3.024468e+13   
33.0   0.000000e+00   0.000000e+00   0.000000e+00  8.634690e-12  2.240620e+14   
35.0   0.000000e+00   0.000000e+00   0.000000e+00  6.017186e-12  1.659921e+15   
37.0   0.000000e+00   0.000000e+00   0.000000e+00  4.269628e-12  1.229721e+16   
39.0   0.000000e+00   0.000000e+00   0.000000e+00  3.079316e-12  9.110157e+16   
41.0   0.000000e+00   0.000000e+00   0.000000e+00  2.253808e-12  6.749088e+17   
43.0   0.000000e+00   0.000000e+00   0.000000e+00  1.671863e-12  4.999934e+18   
45.0   0.000000e+00   0.000000e+00   0.000000e+00  1.255465e-12  3.704107e+19   
47.0   0.000000e+00   0.000000e+00   0.000000e+00  9.534326e-13  2.744117e+20   
49.0   0.000000e+00   0.000000e+00   0.000000e+00  7.315970e-13  2.032927e+21   
51.0   0.000000e+00   0.000000e+00   0.000000e+00  5.667732e-13  1.506056e+22   
53.0   0.000000e+00   0.000000e+00   0.000000e+00  4.429954e-13  1.115733e+23   
55.0   0.000000e+00   0.000000e+00   0.000000e+00  3.491172e-13  8.265697e+23   
57.0   0.000000e+00   0.000000e+00   0.000000e+00  2.772572e-13  6.123485e+24   
59.0   0.000000e+00   0.000000e+00   0.000000e+00  2.217766e-13  4.536468e+25   
61.0   0.000000e+00   0.000000e+00   0.000000e+00  1.785963e-13  3.360757e+26   
63.0   0.000000e+00   0.000000e+00   0.000000e+00  1.447352e-13  2.489753e+27   
65.0   0.000000e+00   0.000000e+00   0.000000e+00  1.179936e-13  1.844486e+28   
67.0   0.000000e+00   0.000000e+00   0.000000e+00  9.673335e-14  1.366453e+29   
69.0   0.000000e+00   0.000000e+00   0.000000e+00  7.972459e-14  1.012311e+30   
71.0   0.000000e+00   0.000000e+00   0.000000e+00  6.603615e-14  7.499512e+30   
73.0   0.000000e+00   0.000000e+00   0.000000e+00  5.495791e-14  5.555872e+31   
75.0   0.000000e+00   0.000000e+00   0.000000e+00  4.594433e-14  4.115962e+32   
77.0   0.000000e+00   0.000000e+00   0.000000e+00  3.857353e-14  3.049233e+33   
79.0   0.000000e+00   0.000000e+00   0.000000e+00  3.251713e-14  2.258966e+34   
81.0   0.000000e+00   0.000000e+00   0.000000e+00  2.751799e-14  1.673513e+35   
83.0   0.000000e+00   0.000000e+00   0.000000e+00  2.337356e-14  1.239790e+36   
85.0   0.000000e+00   0.000000e+00   0.000000e+00  1.992344e-14  9.184750e+36   
87.0   0.000000e+00   0.000000e+00   0.000000e+00  1.703991e-14  6.804348e+37   
89.0   0.000000e+00   0.000000e+00   0.000000e+00  1.462079e-14  5.040873e+38   
91.0   0.000000e+00   0.000000e+00   0.000000e+00  1.258391e-14  3.734435e+39   
93.0   0.000000e+00   0.000000e+00   0.

In [27]:
for col, dn in TLDdistribs_df:
    OutputName = os.path.join('example_outputs', 'TLD_fit_{}.png'.format(col))
    TLDdistribs_df.loc[:,col].to_JPG(OutputName, ylabel='Proportions', title='TLD {}'.format(col))